In [53]:
import nltk
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import math
import pandas as pd

In [43]:
#Предобработка
file = (open('Москва-Петушки.txt', 'r', encoding = 'utf-8')).read()
stop_words = stopwords.words("russian")

In [44]:
#Знаки препинания
punct = re.compile("[^\w\s]")
file_without_punct = re.sub(punct, "", file)

In [45]:
#Регистр
file_without_punct = file_without_punct.lower()

In [46]:
#Токенизация
info_tokenized = word_tokenize(file_without_punct)

In [47]:
#Лемматизация
info_lemmatized = []
for token in info_tokenized:
    lemma = morph.parse(token)[0]
    lemma = lemma.normal_form
    info_lemmatized.append(lemma)

In [48]:
#Фильтрация от стоп-слов
info_filtered = []
for lemma in info_lemmatized:
    if lemma not in stop_words:
        info_filtered.append(lemma)

In [49]:
#Частотный словарь
dictionary = {}
for token in info_filtered:
    if token in dictionary:
        dictionary[token] += 1
    else:
        dictionary[token] = 1

In [51]:
#Сортируем частотный словарь
sorted_dictionary = sorted(dictionary.items(), key=lambda x:x[1], reverse = True)
length = len(info_filtered)

In [52]:
#Создаём частотный список
sorted_list=[]
for word in sorted_dictionary:
    frequency_list = list(word)
    sorted_list.append(frequency_list)

In [54]:
k = 1
for i in sorted_list:
    logfr = math.log(int(i[1]), 10)
    logrank = math.log(k, 10)  # берём десятичные логарифмы для ранга и для частоты
    i.append(k)
    i.append(logfr)
    i.append(logrank)
    k += 1

In [59]:
#Экспорт статистики в файл
frame = pd.DataFrame(sorted_list, columns=['Word', 'Frequency', 'Rank', 'Logarithm of Frequency', 'Logarithm of rank'])
frame.to_csv('Frequency Dictionary.csv',encoding='windows-1251', sep=';', index=False)

In [60]:
#Биграммы и триграммы
bigrams = nltk.collocations.BigramAssocMeasures()
trigrams = nltk.collocations.TrigramAssocMeasures()

bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(info_filtered)
trigramFinder = nltk.collocations.TrigramCollocationFinder.from_words(info_filtered)

# bigrams
bigram_freq = bigramFinder.ngram_fd.items()
bigramFreqTable = pd.DataFrame(list(bigram_freq), columns=['bigram','freq']).sort_values(by='freq', ascending=False)
bigramFreqTable.to_csv('Bigrams.csv',encoding='windows-1251', sep=';', index=False)

# trigrams
trigram_freq = trigramFinder.ngram_fd.items()
trigramFreqTable = pd.DataFrame(list(trigram_freq), columns=['trigram','freq']).sort_values(by='freq', ascending=False)
trigramFreqTable.to_csv('Trigrams.csv',encoding='windows-1251', sep=';', index=False)

In [61]:
# t-test
bigramTtable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.student_t)), columns=['bigram','T-test']).sort_values(by='T-test', ascending=False)
trigramTtable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.student_t)), columns=['trigram','T-test']).sort_values(by='T-test', ascending=False)
bigramTtable = bigramTtable.head(100)
trigramTtable = trigramTtable.head(100) #вычленяем первые 100 лучших би- и триграммов
bigramTtable.to_csv('Bigrams-T-test-100.csv',encoding='windows-1251', sep=';', index=False) #экспортируем в файл
trigramTtable.to_csv('Trigrams-T-test-100.csv',encoding='windows-1251', sep=';', index=False) #экспортируем в файл

# chi-squared
bigramCHItable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.chi_sq)), columns=['bigram','Chi-squared']).sort_values(by='Chi-squared', ascending=False)
trigramCHItable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.chi_sq)), columns=['trigram','Chi-squared']).sort_values(by='Chi-squared', ascending=False)
bigramCHItable = bigramCHItable.head(100)
trigramCHItable = trigramCHItable.head(100) #вычленяем первые 100 лучших би- и триграммов
bigramCHItable.to_csv('Bigrams-CHI-test-100.csv',encoding='windows-1251', sep=';', index=False) #экспортируем в файл
trigramCHItable.to_csv('Trigrams-CHI-test-100.csv',encoding='windows-1251', sep=';', index=False) #экспортируем в файл

# log-likelihood
bigramlikelihoodtable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.likelihood_ratio)), columns=['bigram','Log-Likelihood']).sort_values(by='Log-Likelihood', ascending=False)
trigramlikelihoodtable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.likelihood_ratio)), columns=['trigram','Log-Likelihood']).sort_values(by='Log-Likelihood', ascending=False)
bigramlikelihoodtable = bigramlikelihoodtable.head(100)
trigramlikelihoodtable = trigramlikelihoodtable.head(100) #вычленяем первые 100 лучших би- и триграммов
bigramlikelihoodtable.to_csv('Bigrams-likelihood-test-100.csv',encoding='windows-1251', sep=';', index=False) #экспортируем в файл
trigramlikelihoodtable.to_csv('Trigrams-likelihood-test-100.csv',encoding='windows-1251', sep=';', index=False) #экспортируем в файл

# pmi
bigramPMItable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.pmi)), columns=['bigram','PMI']).sort_values(by='PMI', ascending=False)
trigramPMItable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.pmi)), columns=['trigram','PMI']).sort_values(by='PMI', ascending=False)
bigramPMItable = bigramPMItable.head(100)
trigramPMItable = trigramPMItable.head(100) #вычленяем первые 100 лучших би- и триграммов
bigramPMItable.to_csv('Bigrams-PMI-test-100.csv',encoding='windows-1251', sep=';', index=False) #экспортируем в файл
trigramPMItable.to_csv('Trigrams-PMI-test-100.csv',encoding='windows-1251', sep=';', index=False) #экспортируем в файл